<a href="https://colab.research.google.com/github/Cgrawingholt/Data_extract/blob/main/LeituraXML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leitura de arquivos XML -
Leitura de Notas Fiscais Eletrônicas de produtos e serviços. Extração de dados e exportação para o excel.

In [ ]:
#Importando biblioteca para leitura do xml
!pip install xmltodict

In [6]:
#Importando bibliotecas
import xmltodict
import pandas as pd
#import numpy as np
import os
from google.colab import drive

In [47]:
#Criando função para ler e extrair informação de NFe

def ler_xml_danfe(nota):
  with open(nota, 'rb') as arquivo:
    documento = xmltodict.parse(arquivo)

  dic_notafiscal = documento['nfeProc']['NFe']['infNFe']

  numero = dic_notafiscal['ide']['cNF']
  valor_total = dic_notafiscal['total']['ICMSTot']['vNF']
  cnpj_vendedor = dic_notafiscal['emit']['CNPJ']
  nome_vendedor = dic_notafiscal['emit']['xNome']
  produtos = dic_notafiscal['det']
  lista_produtos = []
  if type (produtos) == list:
    for produto in produtos:
      nome_produto = produto['prod']['xProd']
      valor_produto = produto['prod']['vProd']
      if 'vDesc' in produto['prod']:
        valor_desconto = produto['prod']['vDesc']
      else:
        valor_desconto = 0
      valor_imposto = produto['imposto']['vTotTrib']
      lista_produtos.append((nome_produto, valor_desconto,valor_produto, valor_imposto))
  else:
    nome_produto = produtos['prod']['xProd']
    valor_produto = produtos['prod']['vProd']
    if 'vDesc' in produtos['prod']:
      valor_desconto = produtos['prod']['vDesc']
    else:
      valor_desconto = 0
    valor_imposto = produtos['imposto']['vTotTrib']
    lista_produtos.append((nome_produto, valor_produto, valor_desconto, valor_imposto))

  resposta = {
    'numero' : [numero],
    'valor_total': [valor_total],
    'cnpj_vendedor': [cnpj_vendedor],
    'nome_vendedor': [nome_vendedor],
    'lista_produtos': [lista_produtos],
  }
  return(resposta)

#Criando função para ler e extrair informação de NFs
def ler_xml_servico(nota):
    with open(nota, 'rb') as arquivo:
        documento = xmltodict.parse(arquivo)
    # print(documento)
    dic_notafiscal = documento['ConsultarNfseResposta']['ListaNfse']['CompNfse']['Nfse']['InfNfse']

    valor_total = dic_notafiscal['Servico']['Valores']['ValorServicos']
    cnpj_vendedor = dic_notafiscal['PrestadorServico']['IdentificacaoPrestador']['Cnpj']
    nome_vendedor = dic_notafiscal['PrestadorServico']['RazaoSocial']
    produtos = dic_notafiscal['Servico']['Discriminacao']
    resposta = {
        'valor_total': [valor_total],
        'cnpj_vendedor': [cnpj_vendedor],
        'nome_vendedor': [nome_vendedor],
        'nome_comprador': [nome_comprador],
        'lista_produtos': [produtos],
    }
    return resposta

lista_arquivos = os.listdir('/content/drive/MyDrive/NFE') # lista os nomes dos arquivos de uma pasta

df_final = pd.DataFrame()
for arquivo in lista_arquivos:
    if 'xml' in arquivo:
      if 'DANFE' in arquivo:
        df = pd.DataFrame.from_dict(ler_xml_danfe(f"/content/drive/MyDrive/NFE/{arquivo}"))

      else:
        df = pd.DataFrame.from_dict(ler_xml_servico(f'/content/drive/MyDrive/NFE/{arquivo}'))

      df_final = df_final.append(df, ignore_index = True)
#print(df_final)
df_final.to_excel('NFs.xlsx', index=False)



<ipython-input-47-6fa5ffa54360>:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df, ignore_index = True)
<ipython-input-47-6fa5ffa54360>:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df, ignore_index = True)
<ipython-input-47-6fa5ffa54360>:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df, ignore_index = True)


In [48]:
display(df_final)

,numero,valor_total,cnpj_vendedor,nome_vendedor,lista_produtos
0,58023555,109.00,12863194000138,GRUDADO EM VOCE ETIQUETAS LTDA,"[(Etiqueta Mini, 0, 25.00, 8.36), (Etiqueta Mi..."
1,02258700,987.35,49323314000971,SOMOS SISTEMAS DE ENSINO SA,"[(KIT PLATAFORMA PAR EF2 7O ANO AL - B2C, 987...."
2,42382101,67.22,62277041000187,DISAL - DISTRIBUIDORES ASSOCIADOS DE LIVROS S.A.,"[(WOMEN WHO CHANGED THE WORLD, 84.03, 16.81, 2..."


In [49]:
#Abrindo a coluna de produtos por itens
df_final = df_final.explode('lista_produtos').reset_index(names='nota')
df_final['numero_item'] = df_final.groupby('nota').cumcount()  + 1
df_final = df_final.rename(columns={'lista_produtos': 'item'})
df_final

,nota,numero,valor_total,cnpj_vendedor,nome_vendedor,item,numero_item
0,0,58023555,109.00,12863194000138,GRUDADO EM VOCE ETIQUETAS LTDA,"(Etiqueta Mini, 0, 25.00, 8.36)",1
1,0,58023555,109.00,12863194000138,GRUDADO EM VOCE ETIQUETAS LTDA,"(Etiqueta Mini, 0, 25.00, 8.36)",2
2,0,58023555,109.00,12863194000138,GRUDADO EM VOCE ETIQUETAS LTDA,"(Inicial, 0, 17.00, 5.69)",3
3,0,58023555,109.00,12863194000138,GRUDADO EM VOCE ETIQUETAS LTDA,"(Inicial, 0, 17.00, 5.69)",4
4,0,58023555,109.00,12863194000138,GRUDADO EM VOCE ETIQUETAS LTDA,"(Etiqueta Mini, 0, 25.00, 8.36)",5
5,1,02258700,987.35,49323314000971,SOMOS SISTEMAS DE ENSINO SA,"(KIT PLATAFORMA PAR EF2 7O ANO AL - B2C, 987.3...",1
6,2,42382101,67.22,62277041000187,DISAL - DISTRIBUIDORES ASSOCIADOS DE LIVROS S.A.,"(WOMEN WHO CHANGED THE WORLD, 84.03, 16.81, 20...",1
